In [2]:
#There code that I used from my references. Libraries used came from sources mainly sources 1,10,17,18
#Most other coding sources listed in reference I also used. Note that there may be miscitations so I listed a reference at the bottom of the code
import os
import numpy as np
import torch
import glob
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

2023-12-11 15:19:10.719765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#code in this box came from sources 3, 5, 7. I have used these tutorials mainly
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')

In [4]:
#code in this box came from source 3, 5, 7. I have used these tutorials mainly
data_transformer = {
    'train': transforms.Compose([#training
        transforms.Resize((150,150)),#pixel 224
        transforms.RandomHorizontalFlip(),#flip
        transforms.ToTensor(),#tensor accept tensor - convert tensor
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])#normalize data
    ]),
    'test':transforms.Compose([#testing
        transforms.Resize((150,150)),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    
}
    


In [5]:
#code came from sources 3,5,7,19. Dataset came from source 6
data_dir = '/Users/andyxiao/COEN140/LuggageDetectionProject'
#data loaders

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transformer[x])for x in['train','test']}

In [6]:
#Load data
#sources/code used 3,5,7,19. Dataset came from source 6
dataloader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)for x in ['train','test']}
dataset_sizes = {x: len(image_datasets[x])for x in['train', 'test']}
print(dataset_sizes)

class_names = image_datasets['train'].classes

{'train': 88, 'test': 22}


In [7]:
#sources/code used 3,5,7. Dataset came from source 6
test_loader=DataLoader(
    torchvision.datasets.ImageFolder('/Users/andyxiao/COEN140/LuggageDetectionProject/test',transform=data_transformer['test']),
    batch_size=9,shuffle=True)
train_loader=DataLoader(
    torchvision.datasets.ImageFolder('/Users/andyxiao/COEN140/LuggageDetectionProject/train',transform=data_transformer['train']),
    batch_size=3,shuffle=True)

In [8]:
#sources/code used 3,5,7,19. Dataset came from source 6
#categorites
root = pathlib.Path('/Users/andyxiao/COEN140/LuggageDetectionProject/train')
classes =sorted([j.name.split('/')[-1]for j in root.iterdir()])
print(classes)

['.DS_Store', 'luggage']


In [9]:
#sources/code used from 3,4, 7. I modified some of the layers, kernel size, and number of layers etc... for both functions
#CNN model with 4 layers and 30 output channels with a kernel size of 3
class CNN(nn.Module):
    def __init__(self,num_classes=2):
        super(CNN,self).__init__()
        #output = w-f+2p/s +1 w= 150, f = 3, P = 1, s =1
        #Random CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3,stride=1,padding=1)
        #shape = 256,12,150,150
        self.bn1 = nn.BatchNorm2d(num_features = 6)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = 3)
        #shape = 256,12,50,50
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3,stride=1,padding=1)
        #shape = 256,24,50,50
        self.bn2 = nn.BatchNorm2d(num_features = 12)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3,stride=1,padding=1)
        #shape = 256,24,50,50
        self.bn3 = nn.BatchNorm2d(num_features = 24)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=30, kernel_size=3,stride=1,padding=1)
        #shape = 256,24,50,50
        self.bn4 = nn.BatchNorm2d(num_features = 30)
        self.relu4 = nn.ReLU()
        
        
        #self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3,stride=1,padding=1)
        #self.bn3 = nn.BatchNorm2d(num_features = 20)
        #self.relu3 = nn.ReLU()
        
        self.fc = nn.Linear(in_features=30*50*50,out_features = num_classes)
    
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        
        output = self.pool(output)
        
        output=self.conv2(output)
        output=self.relu2(output)
        
        output=self.conv3(output)
        output=self.relu3(output)
        
        output=self.conv4(output)
        output=self.bn4(output)
        output=self.relu4(output)
        
        #output=self.conv3(input)
        #output=self.bn3(output)
        #output=self.relu3(output)
        
        
        output=output.view(-1,30*50*50)
        
        output=self.fc(output)

        return output

In [17]:
#sources/code used from sources 3,5,7.
model = CNN(num_classes=2).to(device)

print(model)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): Conv2d(24, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (fc): Linear(in_features=75000, out_features=2, bias=True)
)


In [10]:
#sources/code used 3,5,7.
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

NameError: name 'model' is not defined

In [ ]:
#sources 3,5,7,9
num_epochs=10

In [ ]:
#code came from sources 3,5,7. dataset comes from source 6
train_count = len(glob.glob('/Users/andyxiao/COEN140/LuggageDetectionProject/train'+'/*.jpg'))
test_count = len(glob.glob('/Users/andyxiao/COEN140/LuggageDetectionProject/test'+'/*.jpg'))

In [ ]:
#code came from sources 3,5,7
print(train_count,test_count)

In [133]:
#code/sources came from sources 3,4,5,7,9.
#train model with 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    for phase in['train', 'test']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        running_loss = 0.0 #store loss
        running_correct = 0.0 #store correct
        
        for inputs, labels in dataloader[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)#model on device
            
            optimizer.zero_grad()#clear gradients from previous iteration
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_function(outputs, labels)
                if phase == 'train':
                    loss.backward() #backward pass update the weights after gradient calculation
                    optimizer.step()
                    
            running_loss += loss.item() * inputs.size(0)
            running_correct += torch.sum(preds==labels.data)
        
        epochs_loss = running_loss/dataset_sizes[phase]
        epochs_acc = running_correct.double()/dataset_sizes[phase]
        
        print(f'{phase} Loss: {epochs_loss:.4f} Acc: {epochs_acc:.4f}')
        
            

            

train Loss: 0.0335 Acc: 0.9773
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000
train Loss: 0.0000 Acc: 1.0000
test Loss: 0.0000 Acc: 1.0000


In [134]:
torch.save(model.state_dict(),'/Users/andyxiao/COEN140/LuggageDetectionProject.pth')

In [ ]:
#sources/code used 8,12,14,15,16
from PIL import Image

image_path = ''
image = Image.open(image_path)
input_tensor = data_transformer['test'](image)
input_batch = input_tensor.unsqueeze(0)
with torch.no_grad():
    output = model(input_batch)
_, predicted_class = output.max(1)
class_names = ['luggage']
predicted_class = class_names[predicted_class.item()]
print(f'the predicted class is: {predicted_class}')

if(predicted_class=='luggage'):
    AirportVerify(image_path)
    print("Baggage is checked successfully and will be sent for flight verification")
else:
    print("This is not an acceptable baggage")

In [187]:
#sources/code used 8,12,14,15,16,20
def AirportVerify(img):
    text = pytesseract.image_to_string(img)
    if(text == 'SEA'):
        print("Go head")
    else:
        print("this goes to flight ",text)
    return text

In [178]:
def readcsv():
    import pandas as pd
    file = pd.read_csv('/Users/andyxiao/Downloads/Book 7.txt')
    return file

In [1]:
rd =readcsv()
#rd
#print(rd)
Y = rd.iloc[:,-1]
print("y ", Y)

NameError: name 'readcsv' is not defined

In [ ]:
References:

1.   MISHRA, P. (2023). Chapters 3 and 9. In Pytorch recipes: A problem-solution approach to build, train and deploy neural network models (2nd ed.). essay, APRESS.
2    AarohiSingla, A. (2023, October 13). Image-classification-using-pytorch: Image classification on custom dataset. GitHub. https://github.com/AarohiSingla/Image-Classification-Using-Pytorch
3.   AI-SPECIALS, A.-S., & gaurav67890, gaurav67890. (2020, June 30). Image classification using CNN from scratch in pytorch- part 1 training. YouTube. https://www.youtube.com/watch?v=9OHlgDjaE2I
4.   Androbomb, A. (2019, December 11). Using CNN to classify images w/pytorch. Kaggle. https://www.kaggle.com/code/androbomb/using-cnn-to-classify-images-w-pytorch/notebook
5.   Code With Aarohii, C. W. A. (2023, October 13). Image classification using Pytorch and Convolutional Neural Network. YouTube. https://www.youtube.com/watch?v=cJpwQJp9flU
6.   DataCluster Labs, D. L. (2023, January 19). Suitcase/Luggage Dataset Indoor Object Image. Kaggle. https://www.kaggle.com/datasets/dataclusterlabs/suitcaseluggage-dataset/
7.   gaurav, gaurav. (2020, July 2). Pytorch Tutorials. GitHub. https://github.com/gaurav67890/Pytorch_Tutorials/blob/master/cnn-scratch-training.ipynb
8.   Goodwin, A. (2023, December 7). How to extract text from images using Python. Extract Text From Images With Python. https://pdf.wondershare.com/ocr/extracting-text-from-image-python.html
9.   Google, G. (2023). How Many Epochs. Google search. https://www.google.com/search?q=if%2Bwe%2Bhave%2Babig%2Bdataset%2Bhow%2Bis%2Bepoch%2Bdetermined&oq=if%2Bwe%2Bhave%2Babig%2Bdataset%2Bhow%2Bis%2Bepoch%2Bdetermined&gs_lcrp=EgZjaHJvbWUyBggAEEUYOdIBCTEwMzU3ajBqN6gCALACAA&sourceid=chrome&ie=UTF-8
10.  Harris, C. R., Millman, K. J., van der Walt, S. J., Gommers, R., Virtanen, P., Cournapeau, D., Wieser, E., Taylor, J., Berg, S., Smith, N. J., Kern, R., Picus, M., Hoyer, S., van Kerkwijk, M. H., Brett, M., Haldane, A., del Río, J. F., Wiebe, M., Peterson, P., … Oliphant, T. E. (2020, September 16). Array programming with {NumPy}. Numpy. https://numpy.org/citing-numpy/#:~:text=If%20NumPy%20has%20been%20significant,Array%20programming%20with%20NumPy.
11.  Kang, A. (2016, April 28). Delta introduces innovative baggage tracking process. Delta News Hub. https://news.delta.com/delta-introduces-innovative-baggage-tracking-process
12.  Kouidri, A. (2023, October 10). Master text extraction with MMOCR: A comprehensive guide. Ikomia. https://www.ikomia.ai/blog/easy-text-extraction-using-mmocr
13.  May, T. (2023, March 16). Lufthansa partners with Sita to automate mishandled baggage reflight. News. https://airportindustry-news.com/lufthansa-partners-with-sita-to-automate-mishandled-baggage-reflight/
14.  numbers-parsers, numbers-parsers. (2023, September). Numbers-parser. PyPI. https://pypi.org/project/numbers-parser/
15.  Nurfikri, F. (2022, November). How to build optical character recognition (OCR) in Python. Built In. https://builtin.com/data-science/python-ocr
16.  PyPi, P. (2023). Pytesseract. PyPI. https://pypi.org/project/pytesseract/
17.  Pytorch, P. (2023). Getting started with transforms v2¶. Getting started with transforms v2 - Torchvision main documentation. https://pytorch.org/vision/main/auto_examples/transforms/plot_transforms_getting_started.html#sphx-glr-auto-examples-transforms-plot-transforms-getting-started-py9}, journal={Getting started with transforms v2 - Torchvision main documentation
18.  Pytorch, P. (2023b). Membership available. PyTorch. https://pytorch.org/
19.  Tycomac Codes, T. C. (2021, March 15). How to import your own image data set for your deep learning project (tensorflow/pytorch) | tutorial. YouTube. https://www.youtube.com/watch?v=Y33-wDKQs94 
20.  Real Python, R. P. (2023, October 12). How to check if a python string contains a substring. https://realpython.com/python-string-contains-substring/ 
    
     Copyright 2021 Jon Connell